In [11]:
import math
from qat.lang.AQASM.qftarith import IQFT
from qat.lang.AQASM import Program, H
from qat.lang.AQASM.arithmetic import modular_exp
import qat.lang.AQASM.qftarith as qftarith
import qat.lang.AQASM.classarith as classarith
import random

def is_prime(N):
    for i in range(2, N-1):
        if N % i == 0:
            return False
    else:
        return True
    
def compute_prime_power(N):
    """
    O(L^3) algo to check if N is the power of a prime number
    
    Returns:
        int: u s.t u**b == N if exists, else -1
    """
    L = math.ceil(math.log2(N))
    for b in range(2, L):
        u1 = math.floor(N**(1./b))
        u2 = u1 + 1
        if u1**b == N: return u1
        if u2**b == N: return u2
    return -1

assert(compute_prime_power(13**4) == 169)
assert(compute_prime_power(11*3) == -1)

def perform_continued_fraction_expansion(y, N):
    """
    Computes integers d, r' such that r' < N and |y/N - d/r'| < 2**(n_phase_bits + 1)  
    
    Args:
        y (float): a number 
        N (integer): an integer
        
    Returns:
        integer: r'
    
    """
    a = math.floor(y)
    eps = y - a
    p_prev = a
    a = math.floor(1/eps)
    eps = 1/eps - a
    p = a * p_prev + 1
    q_prev  =1
    q = a
    while True:
        a = math.floor(1/eps)
        eps = 1/eps - a
        p_temp = p
        q_temp = q
        p = a * p + p_prev
        q = a * q + q_prev
        p_prev = p_temp
        q_prev = q_temp
        
        if q_prev < N and q >= N: return q_prev


def make_shor_circ(a, N, n_phase_bits):
    """
    Args:
        a (integer): the integer a
        N (integer): the integer N
        n_phase_bits (int): the number of phase bits
        qpu (QPU): a qpu
    
    Returns:
        Circuit: the circuit corresponding to Shor's algorithm
    """
    L = math.ceil(math.log2(N))
    
    prog = Program()
    phase_reg = prog.qalloc(n_phase_bits)
    data_reg = prog.qalloc(L)
    
    for qb in range(n_phase_bits):
        H(phase_reg[qb])
        
    #for qb in range(L):
    #    X(data_reg[qb])
        
    prog.apply(modular_exp(n_phase_bits, L, a, N), phase_reg, data_reg)
    prog.apply(IQFT(n_phase_bits), phase_reg)
    
    #return prog.to_circ(link=[qftarith])
    return prog.to_circ(link=[classarith])

from qat.core.util import statistics
def perform_shor_qpe(a, N, n_phase_bits, qpu, verbose=False):
    """
    Args:
        a (integer): the integer a
        N (integer): the integer N
        n_phase_bits (int): the number of phase bits
        qpu (QPU): a qpu
        
    Returns:
        float: the fraction (betw 0 and 1)
    """
    circ = make_shor_circ(a, N, n_phase_bits)
    
    if verbose:
        print("Shor circ stats:", statistics(circ))
    
    job = circ.to_job(nbshots=0,
                      qubits=list(range(n_phase_bits)))
    
    res = qpu.submit(job)
    for sample in res:
        if verbose:
            print("%s : %s"%(sample.state.int, sample.probability))
        return sample.state.int / 2**n_phase_bits
        
def compute_order(a, N, n_phase_bits, verbose=False):
    """
    Find lowest positive integer r such that a^r == 1 mod. N
    
    Args:
        a (integer): the integer a
        N (integer): the integer N
    """
    y = perform_shor_qpe(a, N, n_phase_bits, qpu, verbose)
    if verbose:
        print("QPE yields y = %s"%y)
    r = perform_continued_fraction_expansion(y, N)
    
    return r


import numpy as np
def perform_shor(N, qpu, max_n_trials=100, n_phase_bits=None, verbose=False):
    """
    Args:
        N (integer): the number to be factorized
        qpu (QPU): a qpu
    """
    # trivial cases
    if N%2 == 0: return 2
    if is_prime(N): return N
    u = compute_prime_power(N)
    if u != -1: return u
    
    already_tested = []
    n_it = 0
    while n_it < max_n_trials:
        n_it += 1
        #a = random.randint(3, N)
        a = 11
        if a in already_tested:
            continue
        else:
            already_tested.append(a)
        if verbose:
            print("Trying with a = %s"%a)
        g = math.gcd(a, N)
        if g != 1:
            return g
        # else: a and N are coprime
        r = compute_order(a, N, n_phase_bits, verbose)
        if verbose:
            print("Found order r = %s"%r)
            
        if r % 2 == 1 or (a**(r/2)) % N == -1:
            continue
        g1 = math.gcd(a**(r/2)+1, N)
        g2 = math.gcd(a**(r/2)-1, N)
        if N % g1 == 0: return g1
        if N % g2 == 0: return g2
        
        
    raise Exception("did not find factorization!")
    

In [12]:
n_phase_bits = 9
y = 427 / 2**n_phase_bits
N = 21
q = perform_continued_fraction_expansion(y, N)

assert(q==6)

In [13]:
from qat.qpus import get_default_qpu
qpu = get_default_qpu()

res = perform_shor(N, qpu, max_n_trials=100, n_phase_bits=n_phase_bits, verbose=True)

Trying with a = 11
Shor circ stats: {'size': 11, 'gates': {'custom gate': 0, 'H': 18, 'X': 1439, 'CNOT': 1438, 'CCNOT': 4320, 'C-C-CNOT': 2250, 'C-C-X': 682, 'C-C-C-CNOT': 540, 'C-C-C-X': 270, 'C-SWAP': 45, 'C-PH': 36}, 'measure': 0, 'reset': 0, 'logic': 0, 'break': 0, 'remap': 0, 'gate_size': 11038}


KeyboardInterrupt: 

In [ ]:
print(res)

In [14]:
from qat.qpus import get_default_qpu
qpu = get_default_qpu()

N=15
n_phase_bits = 2*4

res = perform_shor(N, qpu, max_n_trials=100, n_phase_bits=n_phase_bits, verbose=True)

Trying with a = 11
Shor circ stats: {'size': 10, 'gates': {'custom gate': 0, 'H': 16, 'X': 1473, 'CNOT': 1472, 'CCNOT': 2432, 'C-C-CNOT': 1344, 'C-C-X': 616, 'C-C-C-CNOT': 320, 'C-C-C-X': 256, 'C-SWAP': 32, 'C-PH': 28}, 'measure': 0, 'reset': 0, 'logic': 0, 'break': 0, 'remap': 0, 'gate_size': 7989}
QPE yields y = 0.0


ZeroDivisionError: float division by zero